In [1]:
import os

In [2]:
%pwd

'd:\\Mouli\\University\\external_works\\Content-Summary\\Content-Summary\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\Mouli\\University\\external_works\\Content-Summary\\Content-Summary'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:

    root_directory: Path
    source_URL: str
    local_data_file: Path
    unzip_directory: Path

In [6]:
from contentSummarizer.constants import *
from contentSummarizer.utils.common import read_yaml, create_directories

In [15]:
class ConfigurationManager:

    def __init__(
            self, 
            config_filepath = CONFIG_FILE_PATH, 
            params_filepath = PARAMS_FILE_PATH
    ):
        self.congif = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.congif.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.congif.data_ingestion

        create_directories([config.root_directory])

        data_ingestion_config = DataIngestionConfig(
            root_directory=config.root_directory,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_directory=config.unzip_directory
        )

        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from contentSummarizer.logging import logger
from contentSummarizer.utils.common import get_size

In [9]:
class DataIngestion:

    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL, 
                filename=self.config.local_data_file
            )
            logger.info(f'{filename} downloaded with the following info: \n {headers}')

        else:
            logger.info(f'File already exists of size {get_size(Path(self.config.local_data_file))}')

    def extract_zip_file(self):
        unzip_path = self.config.unzip_directory
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [16]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2024-03-27 16:21:18,291: INFO: common: YAML file: config\config.yaml loaded successfully]
[2024-03-27 16:21:18,293: INFO: common: YAML file: params.yaml loaded successfully]
[2024-03-27 16:21:18,294: INFO: common: Created directory: artifacts]
[2024-03-27 16:21:18,295: INFO: common: Created directory: artifacts/data_ingestion]
[2024-03-27 16:21:20,248: INFO: 3687911766: artifacts/data_ingestion/data.zip downloaded with the following info: 
 Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 8B9A:0EA0:1A6ED1:1F3F7D:66047FBD
Accept-Ranges: bytes
Date: Wed, 27 Mar 2024 20:21:18 GMT
Via: 1.1 varnish
X-Served-By: cache-chi-kigq8000045-CHI
X-Cac